In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from model import Agent
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

2024-06-18 21:32:24.723768: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-18 21:32:24.724139: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 21:32:24.727143: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-18 21:32:24.769326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-18 21:32:25.523348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
df = pd.read_csv('clean_health_data.csv')
df

,Billing_Amount,Age,admited_days,Gender_Male,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,Blood Type_B-,Blood Type_O+,...,Insurance Provider_Medicare,Insurance Provider_UnitedHealthcare,Admission Type_Emergency,Admission Type_Urgent,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Test Results_Inconclusive,Test Results_Normal
0,18856.281306,0.223684,0.034483,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1
1,33643.327287,0.644737,0.172414,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,0
2,27955.096079,0.828947,0.482759,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,37909.782410,0.197368,1.000000,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
4,14238.317814,0.394737,0.655172,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55495,2650.714952,0.381579,1.000000,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
55496,31457.797307,0.631579,0.275862,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
55497,27620.764717,0.328947,0.931034,0,0,0,0,1,0,0,...,0,1,0,1,1,0,0,0,0,0
55498,32451.092358,0.394737,0.172414,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [3]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.1, random_state =43 )
Y_train

18608    48735.848001
20123    42615.243851
9749      3995.424761
10325    26608.430185
27068    28733.379588
             ...     
18448    30896.731703
40753    28559.161643
51455    24378.326343
19776    29357.089704
14148    33211.426429
Name: Billing_Amount, Length: 49950, dtype: float64

In [5]:
def scale_to_minus1_1(X):
    X_min = np.min(X)
    X_max = np.max(X)
    return -1 + 2 * (X - X_min) / (X_max - X_min)

In [6]:
Y_train = scale_to_minus1_1(Y_train)
Y_train

18608    0.852904
20123    0.629413
9749    -0.780770
10325    0.044932
27068    0.122524
           ...   
18448    0.201518
40753    0.116162
51455   -0.036499
19776    0.145298
14148    0.286038
Name: Billing_Amount, Length: 49950, dtype: float64

In [7]:
X.shape

(55500, 27)

In [8]:
agent = Agent(X.shape[1], 1)

2024-06-18 21:32:27.087508: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-18 21:32:27.091706: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
def calculate_reward(pred, label):
    pred = np.array(pred)
    label = np.array(label)
    return - np.mean((pred-label)**2)

In [12]:
total_rewards = []
episodes = 2000
batch_size = 32

for episode in range(episodes):
    loss = 0 
    for i in range(0, len(X_train), batch_size):
        end = i + batch_size
        states = X_train[i:end]
        states = np.expand_dims(states, axis=2).astype('float32')
        labels = Y_train[i:end]

        actions = agent.get_action(states)
        
        reward = calculate_reward(labels,actions)

        next_states = states

        data = [states, actions, states, reward]
        loss += np.mean(reward)
        agent.train(data)

    if episode % 2 == 0:
        agent.update()
        print("Update!")

    print(f'Epoch {episode}, Average Loss: {-loss / len(X_train)}')



ValueError: Passed in object <KerasVariable shape=(27, 128), dtype=float32, path=dense/kernel> of type 'Variable', not tf.Tensor or tf.Variable or ExtensionType.